Họ và tên: Phạm Đức Thể

MSSV: 19522253

Lớp: DS300.N11

LAB03 - 17/10/2022

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1gt7B_r9K1l0eeD8WV8JuCEjf10PcOPrF?usp=sharing)

# **LAB 03 - KHUYẾN NGHỊ DỰA TRÊN NỘI DUNG**

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

pd.options.display.max_colwidth = 200
sns.set()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
PATH = '/content/drive/MyDrive/TÀI LIỆU HỌC TẬP ĐẠI HỌC 2019-2023/NĂM 4 2022-2023/HỌC KỲ 1 2022 - 2023/DS300.N11 - HỆ KHUYẾN NGHỊ/THỰC HÀNH/LAB/LAB03 - 17 10 2022/dataset'

## [The Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset)

### Read dataset

In [3]:
# Load dữ liệu movies_metadata.csv
movies_metadata = pd.read_csv(PATH+'/movies_metadata.csv')
movies_metadata

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circum...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- in...",...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming t...",...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, ...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home...",...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of His Life!,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a storyline about a cult plays in his head.,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'nam

In [4]:
# Xuất ra overview của 5 bộ phim đầu tiên
movies_metadata[['overview']].head(5)

,overview
0,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circum..."
1,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- in..."
2,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming t..."
3,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, ..."
4,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home..."


In [5]:
# Replace NaN with an empty string
movies_metadata['overview'] = movies_metadata['overview'].fillna('')

### Pre-processing data

In [6]:
# Viết hàm tiền xử lý dữ liệu trên cột overview
def processing_data(data):
  # Chuyển về từ thường
  data = data.lower()
  # Xóa dấu câu, ký tự đặc biệt
  data = re.sub('\W+',' ', data)
  # Xóa khoảng trắng đầu và cuối câu
  data = data.strip()
  # Xóa stopword
  data = ' '.join([word for word in data.split() if word not in stopwords.words("english")])
  # Tách từ
  # word_tokens = word_tokenize(data)
  # ...
  return data

In [7]:
# Viết hàm tiền xử lý dữ liệu trên cột overview
def processing_overview(col):
  for i in tqdm(range(movies_metadata.shape[0])):
    try:
      col[i] = processing_data(col[i])
    except:
      print(f'Value of row {i} is {col[i]}')
  return col

In [8]:
# Xử lý dữ liệu trên cột overview
processing_overview(movies_metadata['overview']) 

  0%|          | 0/45466 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 45466/45466 [06:48<00:00, 111.34it/s]


0        led woody andy toys live happily room andy birthday brings buzz lightyear onto scene afraid losing place andy heart woody plots buzz circumstances separate buzz woody owner duo eventually learns p...
1        siblings judy peter discover enchanted board game opens door magical world unwittingly invite alan adult trapped inside game 26 years living room alan hope freedom finish game proves risky three f...
2        family wedding reignites ancient feud next door neighbors fishing buddies john max meanwhile sultry italian divorcée opens restaurant local bait shop alarming locals worry scare fish away less int...
3                   cheated mistreated stepped women holding breath waiting elusive good man break string less stellar lovers friends confidants vannah bernie glo robin talk determined find better way breathe
4                                                 george banks recovered daughter wedding receives news pregnant george wife nina expecting planning selling home pl

In [9]:
# Sử dụng TfidfVectorizer để chuyển cột overview của các bộ phim về tf-idf và lưu vào biến overview_matrix
vectorizer = TfidfVectorizer(stop_words='english')
overview_matrix = vectorizer.fit_transform(movies_metadata['overview'])

In [10]:
# In ra shape của overview_matrix nếu = (45466, 75827) là đúng.
print(overview_matrix.shape)

(45466, 75801)


In [11]:
# Lấy overview_matrix_new có kích thước bằng 1 nửa kích thước của overview_matrix
overview_matrix_new = overview_matrix[:int(overview_matrix.shape[0]/2),:int(overview_matrix.shape[1]/2)]

In [12]:
print(overview_matrix_new.shape)

(22733, 37900)


In [13]:
# Tính toán cosine giữa các bộ phim với nhau bằng linear_kernel
cosine_sim = linear_kernel(overview_matrix_new, overview_matrix_new)
cosine_sim

array([[0.58098714, 0.        , 0.        , ..., 0.        , 0.01757945,
        0.        ],
       [0.        , 0.61125382, 0.02200543, ..., 0.03636249, 0.        ,
        0.        ],
       [0.        , 0.02200543, 0.46607905, ..., 0.        , 0.00649986,
        0.00636462],
       ...,
       [0.        , 0.03636249, 0.        , ..., 0.6609658 , 0.        ,
        0.        ],
       [0.01757945, 0.        , 0.00649986, ..., 0.        , 0.39788609,
        0.00550112],
       [0.        , 0.        , 0.00636462, ..., 0.        , 0.00550112,
        0.52920617]])

In [14]:
cosine_sim.shape

(22733, 22733)

In [15]:
# Đánh index cho các bộ phim bằng pd.Series() và lưu trong biến mapping
mapping = pd.Series(movies_metadata.index, index=movies_metadata['title']).drop_duplicates()
mapping

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45461
Century of Birthing            45462
Betrayal                       45463
Satan Triumphant               45464
Queerama                       45465
Length: 45466, dtype: int64

In [16]:
# Viết hàm nhận vào index của bộ phim trả về tên của bộ phim
def get_title(index):
  Title = movies_metadata.iloc[index]['title']
  return Title

In [17]:
get_title(4)

'Father of the Bride Part II'

In [18]:
# Viết hàm trả và top 10 bộ phim sẽ đem đi khuyến nghị cho người dùng U. Người dung U đã xem bộ phim có tên là "Father of the Bride Part II"
def get_recommendations(title, cosine_sim=cosine_sim):
  # Get the index of the movie that matches the title
  idx = mapping[title]
  # Get the pairwsie similarity scores of all movies with that movie
  sim_scores = list(enumerate(cosine_sim[idx]))
  # Sort the movies based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  # Get the scores of the 10 most similar movies
  sim_scores = sim_scores[1:11]
  # Get the movie mapping
  movie_mapping = [i[0] for i in sim_scores]
  # Return the top 10 most similar movies
  return movies_metadata['title'].iloc[movie_mapping]

In [19]:
get_recommendations('Father of the Bride Part II')

6306           North to Alaska
13611      The Magic of Méliès
19801                  Babbitt
6571                     Kuffs
5005                   Wendigo
7097        The Out of Towners
6793       Father of the Bride
926      It's a Wonderful Life
13928               Visioneers
5571            All Night Long
Name: title, dtype: object